Week of 2/12: 

Changes: 
* i == 0: correct the code to account for the proper month/strategy when entering during the middle of the month 
* Remove the calculation of the available # of shares to be bought for each asset with the given portfolio value and weights. 

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
uni = yf.download(["069500.KS", "229200.KS",
             "133690.KS", "143850.KS",
             "114260.KS", "153130.KS",
             "157450.KS"
             ], 
            start="2000-01-01", end="2022-12-31",
            interval= '1d', 
            group_by='column')

[*********************100%%**********************]  7 of 7 completed


In [3]:
uni_close = uni.Close
test = uni_close.dropna(thresh = 7)

In [4]:
test.columns = ['KODEX 200', 'KODEX 국고채3년', 'TIGER 미국나스닥100', 'TIGER 미국S&P500선물(H)', 'KODEX 단기채권' , 'TIGER 단기통안채', 'KODEX 코스닥150']

In [23]:
test

,KODEX 200,KODEX 국고채3년,TIGER 미국나스닥100,TIGER 미국S&P500선물(H),KODEX 단기채권,TIGER 단기통안채,KODEX 코스닥150
Date,,,,,,,
2015-10-01,23900.0,55865.0,22310.0,22395.0,101480.0,101445.0,10760.0
2015-10-02,23745.0,55850.0,22205.0,22285.0,101485.0,101445.0,10810.0
2015-10-05,23770.0,55900.0,22475.0,22595.0,101485.0,101450.0,10900.0
2015-10-06,23995.0,55885.0,22560.0,22820.0,101500.0,101455.0,10840.0
2015-10-07,24360.0,55845.0,22520.0,22925.0,101510.0,101470.0,10600.0
...,...,...,...,...,...,...,...
2022-12-23,30235.0,56600.0,62845.0,44400.0,104420.0,101995.0,9780.0
2022-12-26,30280.0,56570.0,62745.0,44680.0,104420.0,101995.0,9965.0
2022-12-27,30495.0,56490.0,62925.0,44915.0,104420.0,102005.0,10110.0


In [5]:
test1 = (test.pct_change()+1)
test1.replace(np.nan, 1, inplace = True)
test1 = test1.reset_index()
test1['Date'] = pd.to_datetime(test1['Date'])

In [6]:
test1.insert(1, 'month', test1.Date.dt.month)
test1.insert(1, 'year', test1.Date.dt.year)

In [7]:
test1.insert(1, 'end of month', (test1.Date.dt.month - test1.Date.shift(-1).dt.month).isin([0,np.nan]))

In [10]:
test1

,Date,end of month,year,month,KODEX 200,KODEX 국고채3년,TIGER 미국나스닥100,TIGER 미국S&P500선물(H),KODEX 단기채권,TIGER 단기통안채,KODEX 코스닥150
0,2015-10-01,True,2015,10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2015-10-02,True,2015,10,0.993515,0.999731,0.995294,0.995088,1.000049,1.000000,1.004647
2,2015-10-05,True,2015,10,1.001053,1.000895,1.012159,1.013911,1.000000,1.000049,1.008326
3,2015-10-06,True,2015,10,1.009466,0.999732,1.003782,1.009958,1.000148,1.000049,0.994495
4,2015-10-07,True,2015,10,1.015212,0.999284,0.998227,1.004601,1.000099,1.000148,0.977860
...,...,...,...,...,...,...,...,...,...,...,...
1766,2022-12-23,True,2022,12,0.982773,0.999117,0.976157,0.982953,1.000287,1.000196,0.960236
1767,2022-12-26,True,2022,12,1.001488,0.999470,0.998409,1.006306,1.000000,1.000000,1.018916
1768,2022-12-27,True,2022,12,1.007100,0.998586,1.002869,1.005260,1.000000,1.000098,1.014551
1769,2022-12-28,True,2022,12,0.988195,0.998849,0.977354,0.990760,1.000527,1.000343,0.971316


In [8]:
ret = test1.set_index("Date").iloc[:,3:]

In [9]:
ret

,KODEX 200,KODEX 국고채3년,TIGER 미국나스닥100,TIGER 미국S&P500선물(H),KODEX 단기채권,TIGER 단기통안채,KODEX 코스닥150
Date,,,,,,,
2015-10-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2015-10-02,0.993515,0.999731,0.995294,0.995088,1.000049,1.000000,1.004647
2015-10-05,1.001053,1.000895,1.012159,1.013911,1.000000,1.000049,1.008326
2015-10-06,1.009466,0.999732,1.003782,1.009958,1.000148,1.000049,0.994495
2015-10-07,1.015212,0.999284,0.998227,1.004601,1.000099,1.000148,0.977860
...,...,...,...,...,...,...,...
2022-12-23,0.982773,0.999117,0.976157,0.982953,1.000287,1.000196,0.960236
2022-12-26,1.001488,0.999470,0.998409,1.006306,1.000000,1.000000,1.018916
2022-12-27,1.007100,0.998586,1.002869,1.005260,1.000000,1.000098,1.014551


In [143]:
stock_w = 0.7
bond_w = 0.3

In [144]:
# Halloween 
""" Without the assumption of starting cash"""

""" Weights """
odd_buy = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.3*stock_w])
odd_hold = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.0*stock_w])

port_list = [10000000000]  # Seed money 
port = port_list[0]

shares = []
mkt_val = []
cash_rem = []
strat = []

# Indicator for whether we have entered the market or not
invested = False

# Start Backtest! 
for i in range(len(test1)):
    
    if invested == False: 
        # We are not yet invested in any assets. All Cash 
        pass
    else: 
        # Now, we assume that we have # of shares, mkt value of each asset (i.e. we are invested)
        print("Print daily return", test1['Date'][i])
        new_mkt_value = mkt_value*(ret.iloc[i,:]) # Record returns over the previous period 
        # NOTE: For daily returns, each asset return should be equal to the returns matrix. But, portfolio value may be unique. 
        port = np.sum(new_mkt_value) + cash_remaining # Record new portfolio value 
        port_list.append(new_mkt_value)

        # Daily (Record daily changes in mkt value, shares, and remaining cash)
        # Remaining cash, shares - constant. Mkt value - changing. 
        # Portfolio mkt value - changing (recorded above). 
        if bool(test1['end of month'][i]) == True:
            #shares.append(s.values)
            cash_rem.append(cash_remaining)
            strat.append(x)
            mkt_val.append(new_mkt_value.values)
            mkt_value = new_mkt_value

    # When to rebalance (Rebalance at last trading day of month)
            
    if i == 0: 
        if bool(test1['end of month'][i]) == False:  # If we start at end of the month
            if test1.month.values[i] in [10,11,12,1,2,3]: # Halloween strategy
                X = odd_buy 
                x = "aggressive"
                strat.append(x)
            else:
                X = odd_hold
                x = "passive"
                strat.append(x)
        else:                                # If we start in the middle of the month
            if test1.month.values[i] in [11,12,1,2,3,4]: # Halloween strategy
                X = odd_buy 
                x = "aggressive"
                strat.append(x)
            else:
                X = odd_hold
                x = "passive"
                strat.append(x)
        
        # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
        mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
        mkt_val.append(mkt_value)
        cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
        cash_rem.append(cash_remaining)
        
        # Tell indicator that we are invested
        invested = True

    else: 
        if bool(test1['end of month'][i]) == False: # Rebalance when end of month
            print("Rebalance", test1["Date"][i])
            # Determine which strategy to use
            if test1.month.values[i] in [10,11,12,1,2,3]: # Halloween strategy
                X = odd_buy 
                x = "aggressive"
                strat.append(x)
            else:
                X = odd_hold
                x = "passive"
                strat.append(x)
            # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
            #s = np.floor_divide((X*port),test1.iloc[i,4:]) # Determine number of shares for each asset! 
            #shares.append(s.values)
            mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
            mkt_val.append(mkt_value)
            cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
            cash_rem.append(cash_remaining)


Print daily return 2015-10-02 00:00:00
Print daily return 2015-10-05 00:00:00
Print daily return 2015-10-06 00:00:00
Print daily return 2015-10-07 00:00:00
Print daily return 2015-10-08 00:00:00
Print daily return 2015-10-12 00:00:00
Print daily return 2015-10-13 00:00:00
Print daily return 2015-10-14 00:00:00
Print daily return 2015-10-15 00:00:00
Print daily return 2015-10-16 00:00:00
Print daily return 2015-10-19 00:00:00
Print daily return 2015-10-20 00:00:00
Print daily return 2015-10-21 00:00:00
Print daily return 2015-10-22 00:00:00
Print daily return 2015-10-23 00:00:00
Print daily return 2015-10-26 00:00:00
Print daily return 2015-10-27 00:00:00
Print daily return 2015-10-28 00:00:00
Print daily return 2015-10-29 00:00:00
Print daily return 2015-10-30 00:00:00
Rebalance 2015-10-30 00:00:00
Print daily return 2015-11-02 00:00:00
Print daily return 2015-11-03 00:00:00
Print daily return 2015-11-04 00:00:00
Print daily return 2015-11-05 00:00:00
Print daily return 2015-11-06 00:0

In [145]:
res_mkt_val = pd.DataFrame(mkt_val)
res_mkt_val.columns = ret.columns + ' mkt val'
res_mkt_val.index = ret.index[-len(res_mkt_val):]
res_mkt_val['cash_rem'] = cash_rem
res_mkt_val['port_val'] = res_mkt_val.iloc[:, :].sum(axis=1)
res_mkt_val

,KODEX 200 mkt val,KODEX 국고채3년 mkt val,TIGER 미국나스닥100 mkt val,TIGER 미국S&P500선물(H) mkt val,KODEX 단기채권 mkt val,TIGER 단기통안채 mkt val,KODEX 코스닥150 mkt val,cash_rem,port_val
Date,,,,,,,,,
2015-10-01,1.400000e+09,1.000000e+09,2.100000e+09,1.400000e+09,1.000000e+09,1.000000e+09,0.000000e+00,2.100000e+09,1.000000e+10
2015-10-02,1.390921e+09,9.997315e+08,2.090117e+09,1.393123e+09,1.000049e+09,1.000000e+09,0.000000e+00,2.100000e+09,9.973941e+09
2015-10-05,1.392385e+09,1.000627e+09,2.115531e+09,1.412503e+09,1.000049e+09,1.000049e+09,0.000000e+00,2.100000e+09,1.002114e+10
2015-10-06,1.405565e+09,1.000358e+09,2.123532e+09,1.426568e+09,1.000197e+09,1.000099e+09,0.000000e+00,2.100000e+09,1.005632e+10
2015-10-07,1.426946e+09,9.996420e+08,2.119767e+09,1.433132e+09,1.000296e+09,1.000246e+09,0.000000e+00,2.100000e+09,1.008003e+10
...,...,...,...,...,...,...,...,...,...
2022-12-23,2.141365e+09,1.626221e+09,3.151519e+09,2.186487e+09,1.626367e+09,1.626217e+09,3.227657e+09,0.000000e+00,1.558583e+10
2022-12-26,2.144552e+09,1.625359e+09,3.146504e+09,2.200276e+09,1.626367e+09,1.626217e+09,3.288712e+09,0.000000e+00,1.565799e+10
2022-12-27,2.159779e+09,1.623060e+09,3.155531e+09,2.211848e+09,1.626367e+09,1.626376e+09,3.336566e+09,0.000000e+00,1.573953e+10


In [146]:
res_pct = res_mkt_val[res_mkt_val.columns[:]].div(res_mkt_val[res_mkt_val.columns[-1]], axis=0)

In [147]:
res_pct

,KODEX 200 mkt val,KODEX 국고채3년 mkt val,TIGER 미국나스닥100 mkt val,TIGER 미국S&P500선물(H) mkt val,KODEX 단기채권 mkt val,TIGER 단기통안채 mkt val,KODEX 코스닥150 mkt val,cash_rem,port_val
Date,,,,,,,,,
2015-10-01,0.140000,0.100000,0.210000,0.140000,0.100000,0.100000,0.000000,0.210000,1.0
2015-10-02,0.139455,0.100234,0.209558,0.139676,0.100266,0.100261,0.000000,0.210549,1.0
2015-10-05,0.138945,0.099852,0.211107,0.140952,0.099794,0.099794,0.000000,0.209557,1.0
2015-10-06,0.139769,0.099476,0.211164,0.141858,0.099460,0.099450,0.000000,0.208824,1.0
2015-10-07,0.141562,0.099171,0.210294,0.142175,0.099235,0.099231,0.000000,0.208333,1.0
...,...,...,...,...,...,...,...,...,...
2022-12-23,0.137392,0.104340,0.202204,0.140287,0.104349,0.104339,0.207089,0.000000,1.0
2022-12-26,0.136962,0.103804,0.200952,0.140521,0.103868,0.103859,0.210034,0.000000,1.0
2022-12-27,0.137220,0.103120,0.200484,0.140528,0.103330,0.103331,0.211986,0.000000,1.0


---

In [174]:
def algo(stock_w,bond_w,test1,ret):
    # Halloween 
    """ Without the assumption of starting cash"""

    """ Weights """
    odd_buy = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.3*stock_w])
    odd_hold = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.0*stock_w])

    port_list = [10000000000]  # Seed money 
    port = port_list[0]

    mkt_val = []
    cash_rem = []
    strat = []

    # Indicator for whether we have entered the market or not
    invested = False

    # Start Backtest! 
    for i in range(len(test1)):
        
        if invested == False: # This is the case for the first iteration. 
            # We are not yet invested in any assets. All Cash 
            pass
        else: 
            # Now, we assume that we have # of shares, mkt value of each asset (i.e. we are invested)
            #print("Print daily return", test1['Date'][i])
            new_mkt_value = mkt_value*(ret.iloc[i,:]) # Record returns over the previous period 
            # NOTE: For daily returns, each asset return should be equal to the returns matrix. But, portfolio value may be unique. 
            port = np.sum(new_mkt_value) + cash_remaining # Record new portfolio value 
            port_list.append(new_mkt_value)

            # Daily (Record daily changes in mkt value, shares, and remaining cash)
            # Remaining cash, shares - constant. Mkt value - changing. 
            # Portfolio mkt value - changing (recorded above). 
            if bool(test1['end of month'][i]) == True:
                #shares.append(s.values)
                cash_rem.append(cash_remaining)
                strat.append(x)
                mkt_val.append(new_mkt_value.values)
                mkt_value = new_mkt_value

        # When to rebalance (Rebalance at last trading day of month)
                
        if i == 0:  # On the first iteration, when we aren't invested in anything. 
            if bool(test1['end of month'][i]) == False:  # If we start at end of the month
                if test1.month.values[i] in [10,11,12,1,2,3]: # Halloween strategy
                    X = odd_buy 
                    x = "aggressive"
                    strat.append(x)
                else:
                    X = odd_hold
                    x = "passive"
                    strat.append(x)
            else:                                # If we start in the middle of the month
                if test1.month.values[i] in [11,12,1,2,3,4]: # Halloween strategy
                    X = odd_buy 
                    x = "aggressive"
                    strat.append(x)
                else:
                    X = odd_hold
                    x = "passive"
                    strat.append(x)
            
            # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
            mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
            mkt_val.append(mkt_value)
            cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
            cash_rem.append(cash_remaining)
            
            # Tell indicator that we are invested
            invested = True

        else: 
            if bool(test1['end of month'][i]) == False: # Rebalance when end of month
                #print("Rebalance", test1["Date"][i])
                # Determine which strategy to use
                if test1.month.values[i] in [10,11,12,1,2,3]: # Halloween strategy
                    X = odd_buy 
                    x = "aggressive"
                    strat.append(x)
                else:
                    X = odd_hold
                    x = "passive"
                    strat.append(x)
                # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
                #s = np.floor_divide((X*port),test1.iloc[i,4:]) # Determine number of shares for each asset! 
                #shares.append(s.values)
                mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
                mkt_val.append(mkt_value)
                cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
                cash_rem.append(cash_remaining)
    res_mkt_val = pd.DataFrame(mkt_val)
    res_mkt_val.columns = ret.columns + ' mkt val'
    res_mkt_val.index = ret.index[-len(res_mkt_val):]
    res_mkt_val['cash_rem'] = cash_rem
    res_mkt_val['port_val'] = res_mkt_val.iloc[:, :].sum(axis=1)

    res_pct = res_mkt_val[res_mkt_val.columns[:]].div(res_mkt_val[res_mkt_val.columns[-1]], axis=0)
    return res_mkt_val,res_pct

In [14]:
odd_stock_w = 0.7
odd_bond_w = 0.3
even_stock_w = 0.5
even_bond_w = 0.5
even_passive_stock_w = 0.3
even_passive_bond_w = 0.7

In [15]:
odd_buy = np.array([0.2*odd_stock_w, (1/3*odd_bond_w), 0.3*odd_stock_w, 0.2*odd_stock_w, (1/3*odd_bond_w), (1/3*odd_bond_w), 0.3*odd_stock_w])          # Even 11 - Odd 4
odd_hold = np.array([0.2*odd_stock_w, (1/3*odd_bond_w), 0.3*odd_stock_w, 0.2*odd_stock_w, (1/3*odd_bond_w), (1/3*odd_bond_w), 0.0*odd_stock_w])         # Odd 5 - Odd 10
even_buy = np.array([0.2*even_stock_w, (1/3*even_bond_w), 0.4*even_stock_w, 0.4*even_stock_w, (1/3*even_bond_w), (1/3*even_bond_w), 0.0*even_stock_w])  # Odd 11 - Even 4
even_hold = np.array([0.0*even_passive_stock_w, (1/3*even_passive_bond_w), 0.5*even_passive_stock_w, 0.5*even_passive_stock_w, (1/3*even_passive_bond_w), (1/3*even_passive_bond_w), 0.0*even_passive_stock_w])  # Even 5 - Even 10

In [12]:
def algo1(test1,ret):
    # Halloween Adjusted
    """ Without the assumption of starting cash"""

    """ Weights """
    #odd_buy = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.3*stock_w])
    #odd_hold = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.0*stock_w])

    port_list = [10000000000]  # Seed money 
    port = port_list[0]

    mkt_val = []
    cash_rem = []
    strat = []

    # Indicator for whether we have entered the market or not
    invested = False

    # Start Backtest! 
    for i in range(len(test1)):
        
        if invested == False: 
            # We are not yet invested in any assets. All Cash 
            pass
        else: 
            # Now, we assume that we have # of shares, mkt value of each asset (i.e. we are invested)
            #print("Print daily return", test1['Date'][i])
            new_mkt_value = mkt_value*(ret.iloc[i,:]) # Record returns over the previous period 
            # NOTE: For daily returns, each asset return should be equal to the returns matrix. But, portfolio value may be unique. 
            port = np.sum(new_mkt_value) + cash_remaining # Record new portfolio value 
            port_list.append(new_mkt_value)

            # Daily (Record daily changes in mkt value, shares, and remaining cash)
            # Remaining cash, shares - constant. Mkt value - changing. 
            # Portfolio mkt value - changing (recorded above). 
            if bool(test1['end of month'][i]) == True:
                #shares.append(s.values)
                cash_rem.append(cash_remaining)
                strat.append(x)
                mkt_val.append(new_mkt_value.values)
                mkt_value = new_mkt_value

        # When to rebalance (Rebalance at last trading day of month)
                
        if i == 0: 
            if bool(test1['end of month'][i]) == False:  # If we start at end of the month
                if ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 1)): # Halloween strategy
                    X = odd_buy 
                    x = "aggressive"
                    strat.append(x)
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                    strat.append(x)
                elif ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                    strat.append(x)
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"
                    strat.append(x)
            else:                                # If we start in the middle of the month
                if ((test1.month.values[i] in [11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3,4]) & (test1.year.values[i]%2 == 1)): 
                    X = odd_buy 
                    x = "odd aggressive"
                    strat.append(x)
                elif (test1.month.values[i] in [5,6,7,8,9,10]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                    strat.append(x)
                elif ((test1.month.values[i] in [11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3,4]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                    strat.append(x)
                elif (test1.month.values[i] in [5,6,7,8,9,10]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"
                    strat.append(x)
            
            # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
            mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
            mkt_val.append(mkt_value)
            cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
            cash_rem.append(cash_remaining)
            
            # Tell indicator that we are invested
            invested = True

        else: 
            if bool(test1['end of month'][i]) == False: # Rebalance when end of month
                #print("Rebalance", test1["Date"][i])
                # Determine which strategy to use
                if ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 1)): # Halloween strategy
                    X = odd_buy 
                    x = "aggressive"
                    strat.append(x)
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                    strat.append(x)
                elif ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                    strat.append(x)
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"
                    strat.append(x)

                    
                # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
                #s = np.floor_divide((X*port),test1.iloc[i,4:]) # Determine number of shares for each asset! 
                #shares.append(s.values)
                mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
                mkt_val.append(mkt_value)
                cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
                cash_rem.append(cash_remaining)
    res_mkt_val = pd.DataFrame(mkt_val)
    res_mkt_val.columns = ret.columns + ' mkt val'
    res_mkt_val.index = ret.index[-len(res_mkt_val):]
    res_mkt_val['cash_rem'] = cash_rem
    res_mkt_val['port_val'] = res_mkt_val.iloc[:, :].sum(axis=1)

    res_pct = res_mkt_val[res_mkt_val.columns[:]].div(res_mkt_val[res_mkt_val.columns[-1]], axis=0)
    return res_mkt_val,res_pct

In [16]:
y1 = algo1( test1, ret)

In [17]:
y1[0]

,KODEX 200 mkt val,KODEX 국고채3년 mkt val,TIGER 미국나스닥100 mkt val,TIGER 미국S&P500선물(H) mkt val,KODEX 단기채권 mkt val,TIGER 단기통안채 mkt val,KODEX 코스닥150 mkt val,cash_rem,port_val
Date,,,,,,,,,
2015-10-01,1.400000e+09,1.000000e+09,2.100000e+09,1.400000e+09,1.000000e+09,1.000000e+09,0.000000e+00,2.100000e+09,1.000000e+10
2015-10-02,1.390921e+09,9.997315e+08,2.090117e+09,1.393123e+09,1.000049e+09,1.000000e+09,0.000000e+00,2.100000e+09,9.973941e+09
2015-10-05,1.392385e+09,1.000627e+09,2.115531e+09,1.412503e+09,1.000049e+09,1.000049e+09,0.000000e+00,2.100000e+09,1.002114e+10
2015-10-06,1.405565e+09,1.000358e+09,2.123532e+09,1.426568e+09,1.000197e+09,1.000099e+09,0.000000e+00,2.100000e+09,1.005632e+10
2015-10-07,1.426946e+09,9.996420e+08,2.119767e+09,1.433132e+09,1.000296e+09,1.000246e+09,0.000000e+00,2.100000e+09,1.008003e+10
...,...,...,...,...,...,...,...,...,...
2022-12-23,2.130665e+09,1.618095e+09,3.135772e+09,2.175562e+09,1.618241e+09,1.618091e+09,3.211530e+09,0.000000e+00,1.550796e+10
2022-12-26,2.133836e+09,1.617238e+09,3.130782e+09,2.189282e+09,1.618241e+09,1.618091e+09,3.272280e+09,0.000000e+00,1.557975e+10
2022-12-27,2.148987e+09,1.614950e+09,3.139763e+09,2.200796e+09,1.618241e+09,1.618250e+09,3.319894e+09,0.000000e+00,1.566088e+10


In [22]:
test1

,Date,end of month,year,month,KODEX 200,KODEX 국고채3년,TIGER 미국나스닥100,TIGER 미국S&P500선물(H),KODEX 단기채권,TIGER 단기통안채,KODEX 코스닥150
0,2015-10-01,True,2015,10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2015-10-02,True,2015,10,0.993515,0.999731,0.995294,0.995088,1.000049,1.000000,1.004647
2,2015-10-05,True,2015,10,1.001053,1.000895,1.012159,1.013911,1.000000,1.000049,1.008326
3,2015-10-06,True,2015,10,1.009466,0.999732,1.003782,1.009958,1.000148,1.000049,0.994495
4,2015-10-07,True,2015,10,1.015212,0.999284,0.998227,1.004601,1.000099,1.000148,0.977860
...,...,...,...,...,...,...,...,...,...,...,...
1766,2022-12-23,True,2022,12,0.982773,0.999117,0.976157,0.982953,1.000287,1.000196,0.960236
1767,2022-12-26,True,2022,12,1.001488,0.999470,0.998409,1.006306,1.000000,1.000000,1.018916
1768,2022-12-27,True,2022,12,1.007100,0.998586,1.002869,1.005260,1.000000,1.000098,1.014551
1769,2022-12-28,True,2022,12,0.988195,0.998849,0.977354,0.990760,1.000527,1.000343,0.971316


In [19]:
def algo2(test1,ret):
    # Halloween Adjusted
    """ Without the assumption of starting cash"""

    """ Weights """
    #odd_buy = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.3*stock_w])
    #odd_hold = np.array([0.2*stock_w, (1/3*bond_w), 0.3*stock_w, 0.2*stock_w, (1/3*bond_w), (1/3*bond_w), 0.0*stock_w])

    port_list = [10000000000]  # Seed money 
    port = port_list[0]

    mkt_val = []
    cash_rem = []
    strat = []

    # Indicator for whether we have entered the market or not
    invested = False

    # Start Backtest! 
    for i in range(len(test1)):
        
        if invested == False: 
            # We are not yet invested in any assets. All Cash 
            pass
        else: 
            # Now, we assume that we have # of shares, mkt value of each asset (i.e. we are invested)
            #print("Print daily return", test1['Date'][i])
            new_mkt_value = mkt_value*(ret.iloc[i,:]) # Record returns over the previous period 
            # NOTE: For daily returns, each asset return should be equal to the returns matrix. But, portfolio value may be unique. 
            port = np.sum(new_mkt_value) + cash_remaining # Record new portfolio value 
            port_list.append(new_mkt_value)

            # Daily (Record daily changes in mkt value, shares, and remaining cash)
            # Remaining cash, shares - constant. Mkt value - changing. 
            # Portfolio mkt value - changing (recorded above). 
            if bool(test1['end of month'][i]) == True:
                #shares.append(s.values)
                cash_rem.append(cash_remaining)
                strat.append(x)
                mkt_val.append(new_mkt_value.values)
                mkt_value = new_mkt_value

        # When to rebalance (Rebalance at last trading day of month)
                
        if i == 0: 
            if bool(test1['end of month'][i]) == False:  # If we start at end of the month
                if ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 1)): # Halloween strategy
                    X = odd_buy 
                    x = "aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                elif ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"
            else:                                # If we start in the middle of the month
                if ((test1.month.values[i] in [11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3,4]) & (test1.year.values[i]%2 == 1)): 
                    X = odd_buy 
                    x = "odd aggressive"
                elif (test1.month.values[i] in [5,6,7,8,9,10]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                elif ((test1.month.values[i] in [11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3,4]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                elif (test1.month.values[i] in [5,6,7,8,9,10]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"
                    
            strat.append(x)
            # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
            mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
            mkt_val.append(mkt_value)
            cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
            cash_rem.append(cash_remaining)
            
            # Tell indicator that we are invested
            invested = True

        else: 
            if bool(test1['end of month'][i]) == False: # Rebalance when end of month
                #print("Rebalance", test1["Date"][i])
                # Determine which strategy to use
                if ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 1)): # Halloween strategy
                    X = odd_buy 
                    x = "aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                elif ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"

                strat.append(x)
                # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
                #s = np.floor_divide((X*port),test1.iloc[i,4:]) # Determine number of shares for each asset! 
                #shares.append(s.values)
                mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
                mkt_val.append(mkt_value)
                cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
                cash_rem.append(cash_remaining)

                
    res_mkt_val = pd.DataFrame(mkt_val)
    res_mkt_val.columns = ret.columns + ' mkt val'
    res_mkt_val.index = ret.index[-len(res_mkt_val):]
    res_mkt_val['cash_rem'] = cash_rem
    res_mkt_val['port_val'] = res_mkt_val.iloc[:, :].sum(axis=1)

    res_pct = res_mkt_val[res_mkt_val.columns[:]].div(res_mkt_val[res_mkt_val.columns[-1]], axis=0)
    return res_mkt_val,res_pct

In [20]:
y1 = algo2( test1, ret)

In [21]:
y1[0]

,KODEX 200 mkt val,KODEX 국고채3년 mkt val,TIGER 미국나스닥100 mkt val,TIGER 미국S&P500선물(H) mkt val,KODEX 단기채권 mkt val,TIGER 단기통안채 mkt val,KODEX 코스닥150 mkt val,cash_rem,port_val
Date,,,,,,,,,
2015-10-01,1.400000e+09,1.000000e+09,2.100000e+09,1.400000e+09,1.000000e+09,1.000000e+09,0.000000e+00,2.100000e+09,1.000000e+10
2015-10-02,1.390921e+09,9.997315e+08,2.090117e+09,1.393123e+09,1.000049e+09,1.000000e+09,0.000000e+00,2.100000e+09,9.973941e+09
2015-10-05,1.392385e+09,1.000627e+09,2.115531e+09,1.412503e+09,1.000049e+09,1.000049e+09,0.000000e+00,2.100000e+09,1.002114e+10
2015-10-06,1.405565e+09,1.000358e+09,2.123532e+09,1.426568e+09,1.000197e+09,1.000099e+09,0.000000e+00,2.100000e+09,1.005632e+10
2015-10-07,1.426946e+09,9.996420e+08,2.119767e+09,1.433132e+09,1.000296e+09,1.000246e+09,0.000000e+00,2.100000e+09,1.008003e+10
...,...,...,...,...,...,...,...,...,...
2022-12-23,2.130665e+09,1.618095e+09,3.135772e+09,2.175562e+09,1.618241e+09,1.618091e+09,3.211530e+09,0.000000e+00,1.550796e+10
2022-12-26,2.133836e+09,1.617238e+09,3.130782e+09,2.189282e+09,1.618241e+09,1.618091e+09,3.272280e+09,0.000000e+00,1.557975e+10
2022-12-27,2.148987e+09,1.614950e+09,3.139763e+09,2.200796e+09,1.618241e+09,1.618250e+09,3.319894e+09,0.000000e+00,1.566088e+10


In [150]:
portfolio_df = res_mkt_val.copy().reset_index()

In [151]:
# Calculate the portfolio percentage daily return and replace NaNs with zeros
portfolio_df['Portfolio Daily Return [%]'] = portfolio_df['port_val'].pct_change(1) * 100 
portfolio_df.replace(np.nan, 0, inplace = True)

In [152]:
portfolio_df

,Date,KODEX 200 mkt val,KODEX 국고채3년 mkt val,TIGER 미국나스닥100 mkt val,TIGER 미국S&P500선물(H) mkt val,KODEX 단기채권 mkt val,TIGER 단기통안채 mkt val,KODEX 코스닥150 mkt val,cash_rem,port_val,Portfolio Daily Return [%]
0,2015-10-01,1.400000e+09,1.000000e+09,2.100000e+09,1.400000e+09,1.000000e+09,1.000000e+09,0.000000e+00,2.100000e+09,1.000000e+10,0.000000
1,2015-10-02,1.390921e+09,9.997315e+08,2.090117e+09,1.393123e+09,1.000049e+09,1.000000e+09,0.000000e+00,2.100000e+09,9.973941e+09,-0.260587
2,2015-10-05,1.392385e+09,1.000627e+09,2.115531e+09,1.412503e+09,1.000049e+09,1.000049e+09,0.000000e+00,2.100000e+09,1.002114e+10,0.473260
3,2015-10-06,1.405565e+09,1.000358e+09,2.123532e+09,1.426568e+09,1.000197e+09,1.000099e+09,0.000000e+00,2.100000e+09,1.005632e+10,0.351008
4,2015-10-07,1.426946e+09,9.996420e+08,2.119767e+09,1.433132e+09,1.000296e+09,1.000246e+09,0.000000e+00,2.100000e+09,1.008003e+10,0.235772
...,...,...,...,...,...,...,...,...,...,...,...
1766,2022-12-23,2.141365e+09,1.626221e+09,3.151519e+09,2.186487e+09,1.626367e+09,1.626217e+09,3.227657e+09,0.000000e+00,1.558583e+10,-1.806531
1767,2022-12-26,2.144552e+09,1.625359e+09,3.146504e+09,2.200276e+09,1.626367e+09,1.626217e+09,3.288712e+09,0.000000e+00,1.565799e+10,0.462945
1768,2022-12-27,2.159779e+09,1.623060e+09,3.155531e+09,2.211848e+09,1.626367e+09,1.626376e+09,3.336566e+09,0.000000e+00,1.573953e+10,0.520763
1769,2022-12-28,2.134282e+09,1.621193e+09,3.084070e+09,2.191411e+09,1.627224e+09,1.626934e+09,3.240858e+09,0.000000e+00,1.552597e+10,-1.356801


---

In [157]:
def simulation_engine(stock_w,bond_w,rf):
    portfolio_df, pct_ret = algo(stock_w,bond_w, test1, ret)
    # Calculate the portfolio percentage daily return and replace NaNs with zeros
    portfolio_df['Portfolio Daily Return [%]'] = portfolio_df['port_val'].pct_change(1) * 100 
    portfolio_df.replace(np.nan, 0, inplace = True)
    portfolio_df = portfolio_df.reset_index()
    
    return_on_investment = ((portfolio_df['port_val'][-1:] - 
                             portfolio_df['port_val'][0])/ 
                             portfolio_df['port_val'][0]) * 100
    
    portfolio_daily_return_df = portfolio_df['Portfolio Daily Return [%]']/100

    annualized_return = np.power(1 + portfolio_daily_return_df.mean(), 252) - 1
    annualized_volatility = portfolio_daily_return_df.std() * np.sqrt(252)
    sharpe_ratio = (annualized_return - 0.01) / annualized_volatility

    sharpe_ratio = (annualized_return - rf)/annualized_volatility 
    return annualized_return, annualized_volatility, sharpe_ratio,return_on_investment.values[0]

In [159]:
portfolio_metrics = simulation_engine(stock_w,bond_w, 0.01)

print('Expected Portfolio Annual Return = {:.2f}%'.format(portfolio_metrics[0] * 100))
print('Portfolio Standard Deviation (Volatility) = {:.2f}%'.format(portfolio_metrics[1] * 100))
print('Sharpe Ratio = {:.2f}'.format(portfolio_metrics[2]))
print('Return on Investment = {:.2f}%'.format(portfolio_metrics[3]))

Expected Portfolio Annual Return = 6.91%
Portfolio Standard Deviation (Volatility) = 10.72%
Sharpe Ratio = 0.55
Return on Investment = 53.65%


In [168]:
sim_runs = 100
n = 2

rf = 0.01

x = np.linspace(0,1,sim_runs).reshape(sim_runs, 1)
y = (1 - x).reshape(sim_runs, 1)
weights = np.hstack((x, y))
sharpe_ratio_runs = np.zeros(sim_runs)
expected_portfolio_returns_runs = np.zeros(sim_runs)
volatility_runs = np.zeros(sim_runs)
return_on_investment_runs = np.zeros(sim_runs)


In [169]:
for i in range(sim_runs):
    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], return_on_investment_runs[i] = simulation_engine(x[i,:][0],y[i,:][0], rf)
    print("Simulation Run = {}".format(i))   
    print("Weights = {}, ROI = %{:.2f},Sharpe Ratio = {:.2f}".format(weights[i].round(3),return_on_investment_runs[i],sharpe_ratio_runs[i]))   
    print('\n')


Simulation Run = 0
Weights = [0. 1.], ROI = %1.53,Sharpe Ratio = -0.75


Simulation Run = 1
Weights = [0.01 0.99], ROI = %2.19,Sharpe Ratio = -0.66


Simulation Run = 2
Weights = [0.02 0.98], ROI = %2.85,Sharpe Ratio = -0.56


Simulation Run = 3
Weights = [0.03 0.97], ROI = %3.51,Sharpe Ratio = -0.46


Simulation Run = 4
Weights = [0.04 0.96], ROI = %4.18,Sharpe Ratio = -0.35


Simulation Run = 5
Weights = [0.051 0.949], ROI = %4.85,Sharpe Ratio = -0.25


Simulation Run = 6
Weights = [0.061 0.939], ROI = %5.52,Sharpe Ratio = -0.17


Simulation Run = 7
Weights = [0.071 0.929], ROI = %6.20,Sharpe Ratio = -0.09


Simulation Run = 8
Weights = [0.081 0.919], ROI = %6.88,Sharpe Ratio = -0.02


Simulation Run = 9
Weights = [0.091 0.909], ROI = %7.56,Sharpe Ratio = 0.03


Simulation Run = 10
Weights = [0.101 0.899], ROI = %8.24,Sharpe Ratio = 0.08


Simulation Run = 11
Weights = [0.111 0.889], ROI = %8.93,Sharpe Ratio = 0.13


Simulation Run = 12
Weights = [0.121 0.879], ROI = %9.62,Sharpe Rat

In [170]:
sharpe_ratio_runs.argmax()

99

In [171]:
sharpe_ratio_runs.max()

0.5833086452737594

In [173]:
weights[sharpe_ratio_runs.argmax(), :]

array([1., 0.])

---

In [62]:
%matplotlib inline
import quantstats as qs

In [63]:
# extend pandas functionality with metrics, etc.
qs.extend_pandas()

# fetch the daily returns for a stock
stock = qs.utils.download_returns('META')

# show sharpe ratio
qs.stats.sharpe(stock)

# or using extend_pandas() :)
stock.sharpe()

[*********************100%%**********************]  1 of 1 completed


0.7326506523617843

In [64]:
stock

Date
2012-05-18         NaN
2012-05-21   -0.109861
2012-05-22   -0.089039
2012-05-23    0.032258
2012-05-24    0.032187
                ...   
2024-02-12    0.001688
2024-02-13   -0.018725
2024-02-14    0.028601
2024-02-15    0.022714
2024-02-16   -0.022127
Name: Close, Length: 2956, dtype: float64

In [67]:
qs.reports.html(stock, "SPY")

[*********************100%%**********************]  1 of 1 completed
c:\Users\ewp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\ewp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\ewp\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this